In [1]:
import os 
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd

In [4]:
os.chdir("/work/FAC/FGSE/IDYST/tbeucler/downscaling/mberthier/repos/Downscaling_CM/data")
june=xr.open_dataset('juin_1999_2010.nc')
july=xr.open_dataset('juillet_1999_2010.nc')
august=xr.open_dataset('aout_1999_2010.nc')

In [5]:
lat_bnd = [-0.6, 0.58]
lon_bnd = [-2.6,-1.42]

In [6]:
june=june.sel(rlon=slice(*lon_bnd), rlat=slice(*lat_bnd))
july=july.sel(rlon=slice(*lon_bnd), rlat=slice(*lat_bnd))
august=august.sel(rlon=slice(*lon_bnd), rlat=slice(*lat_bnd))

In [8]:
summer=xr.concat((june,july,august),dim='time')

In [7]:
##Travailler avec un dataset
df = summer.to_dataframe()
df = df.reset_index()

max_precipitation = df.groupby(['rlon', 'rlat'])['TOT_PR'].max()
max_precipitation_index = df.groupby(['rlon', 'rlat'])['TOT_PR'].idxmax()

date_time = df.loc[max_precipitation_index, 'time']
temperature = df.loc[max_precipitation_index, 'T_2M']
humidity = df.loc[max_precipitation_index, 'RELHUM_2M']
#coord_elev = pd.read_csv("coord_elev.csv")
#coord_elev = coord_elev[['rlon', 'rlat', 'alt']]
#alt=coord_elev['alt']

result_df = pd.DataFrame({'rlon': max_precipitation.index.get_level_values('rlon'),
                          'rlat': max_precipitation.index.get_level_values('rlat'),
                          'max_precipitation': max_precipitation.values,
                          'date_time': date_time.values,
                          'temperature': temperature.values,
                          'humidity': humidity.values})
                          #'alt':alt})
print(result_df)

     rlon  rlat  max_precipitation           date_time  temperature   humidity
0   -2.60 -0.60           0.018432 2004-08-10 19:00:00   291.577576  99.001808
1   -2.60 -0.58           0.015870 2000-08-30 14:00:00   291.304352  95.203392
2   -2.60 -0.56           0.012798 2002-06-24 05:00:00   290.130859  92.093353
3   -2.60 -0.54           0.015116 2002-06-24 05:00:00   290.231873  93.203773
4   -2.60 -0.52           0.015977 2002-06-24 05:00:00   290.478271  93.414223
..    ...   ...                ...                 ...          ...        ...
571 -2.14 -0.22           0.013556 2004-06-10 17:00:00   292.835114  86.617958
572 -2.14 -0.20           0.022112 2004-06-10 17:00:00   292.438385  89.811226
573 -2.14 -0.18           0.018337 2001-08-17 08:00:00   289.956482  94.434341
574 -2.14 -0.16           0.021215 2001-06-27 17:00:00   294.000732  87.625595
575 -2.14 -0.14           0.028198 2001-06-27 17:00:00   294.902130  87.124924

[576 rows x 6 columns]


In [ ]:
import pandas as pd

# Convertir en DataFrame
df = summer.to_dataframe()
df = df.reset_index()

# Convertir la colonne 'time' en type datetime
df['time'] = pd.to_datetime(df['time'])

# Ajouter une colonne 'week' représentant le numéro de semaine
df['week'] = df['time'].dt.strftime('%Y-%U')

# Grouper par semaine et trouver les précipitations maximales par semaine
max_precipitation_week = df.groupby(['rlon', 'rlat', 'week'])['TOT_PR'].max()

# Réinitialiser l'index
max_precipitation_week = max_precipitation_week.reset_index()

# Créer une liste des précipitations maximales par semaine
max_precipitation_list = max_precipitation_week.groupby(['rlon', 'rlat'])['TOT_PR'].apply(list)

# Fusionner les données pour créer le nouveau DataFrame
result_df = pd.merge(df, max_precipitation_week, on=['rlon', 'rlat', 'week'])

# Ajouter la colonne 'max_precipitation_week' contenant la liste des maxima par semaine
result_df['max_precipitation_week'] = result_df.apply(lambda row: max_precipitation_list[(row['rlon'], row['rlat'])], axis=1)

# Sélectionner les colonnes nécessaires dans le résultat final
result_df = result_df[['rlon', 'rlat', 'max_precipitation', 'date_time', 'temperature', 'humidity', 'max_precipitation_week']]

print(result_df)


In [9]:
result_df.to_csv('dataset_variables_leman.csv')

In [ ]:
##On fait un dataset avec la température 

In [19]:
print(coord_elev)

    rlat  rlon  alt
0  -0.50  -2.4  370
1  -0.48  -2.4  370
2  -0.46  -2.4  370
3  -0.44  -2.4  370
4  -0.42  -2.4  370
..   ...   ...  ...
61 -0.38  -2.3  448
62 -0.36  -2.3  588
63 -0.34  -2.3  590
64 -0.32  -2.3  603
65 -0.30  -2.3  634

[66 rows x 3 columns]


In [1]:
#Obtenir le max par année
june = june['TOT_PR'].groupby('time.year').max(dim='time')
july = july['TOT_PR'].groupby('time.year').max(dim='time')
august = august['TOT_PR'].groupby('time.year').max(dim='time')

summer_monthly_max=xr.concat((june,july,august),dim='time')
summer_monthly_max


NameError: name 'june' is not defined

In [49]:
summer_monthly_max.isel(time=0,year=2)

<xarray.DataArray 'TOT_PR' (rlat: 11, rlon: 6)>
array([[0.00510249, 0.00325536, 0.0031432 , 0.00501673, 0.00643883,
        0.00515982],
       [0.0045597 , 0.00255674, 0.00198622, 0.00395188, 0.00553313,
        0.0039079 ],
       [0.00235594, 0.00157344, 0.0020412 , 0.00300806, 0.00232243,
        0.00303557],
       [0.00167888, 0.0020701 , 0.00240452, 0.00185354, 0.0017637 ,
        0.00218646],
       [0.00181397, 0.00263256, 0.00302736, 0.00210262, 0.00192865,
        0.00224239],
       [0.00774235, 0.00271404, 0.0022566 , 0.00212817, 0.0020623 ,
        0.00199876],
       [0.01111663, 0.00890023, 0.0035778 , 0.00208389, 0.00190766,
        0.0016784 ],
       [0.00876445, 0.00835779, 0.00415122, 0.00224505, 0.00199565,
        0.00157398],
       [0.00439995, 0.00373139, 0.00173194, 0.00212946, 0.00262372,
        0.0037224 ],
       [0.00340597, 0.00159659, 0.0015188 , 0.00188053, 0.0040435 ,
        0.00637863],
       [0.00280604, 0.00212637, 0.00195134, 0.0018684 , 0.00307833,
        0.00477453]], dtype=float32)
Coordinates:
  * rlon     (rlon) float32 -2.4 -2.38 -2.36 -2.34 -2.32 -2.3
  * rlat     (rlat) float32 -0.5 -0.48 -0.46 -0.44 ... -0.36 -0.34 -0.32 -0.3
    year     int64 2001
Attributes:
    standard_name:  precipitation_rate
    long_name:      total precipitation rate
    units:          kg m-2 s-1
    grid_mapping:   rotated_pole
    coordinates:    lon lat

In [11]:
summer_monthly_dataset=summer_monthly_max.to_dataframe()

In [8]:
os.chdir("/work/FAC/FGSE/IDYST/tbeucler/downscaling/mberthier/repos/Downscaling_CM/utils")
import dataset_copy

In [9]:
os.chdir("/work/FAC/FGSE/IDYST/tbeucler/downscaling/mberthier/repos/Downscaling_CM/data")

In [12]:
dataset_copy.save_dataset(summer_monthly_max,'summer_monthly_max_lausanne_etendu')
summer_monthly_dataset.to_csv('summer_monthly.csv')

saving to  ./summer_monthly_max_lausanne_etendu.nc
finished saving
